In [ ]:
import pandas
import matplotlib
from arango import ArangoClient
from arango_datasets import Datasets

In [ ]:
# Connect to database
db = ArangoClient(hosts=...).db(username=..., password=..., verify=True)

# Connect to datasets
datasets = Datasets(db)

print(datasets.dataset_info("OPEN_INTELLIGENCE"))

datasets.load("OPEN_INTELLIGENCE")